In [1]:
# Standard Library Imports
import os
import sys
import timeit
from scipy.optimize import least_squares

# Optimize performance by setting environment variables
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['KERAS_BACKEND'] = 'tensorflow'
from timeit import repeat

# Third-party Library Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import arviz as az
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, multivariate_normal, uniform
from tensorflow.keras.models import load_model
from itertools import product
from numba import jit

# Local Module Imports
sys.path.append('./data/data_generation/')
import tinyDA as tda

from model import Model
# from utils import *

case = "MLDA2"  # Options: "2-step"/"1-step"/"FOM"

# MCMC Parameters
noise        = 0.001
scaling      = 0.05
scaling1     = 1
scaling2     = 1
scaling3     = 1
n_iter       = 600
burnin       = 100
thin         = 1
sub_sampling = [10,5]

# Initialize Parameters
n_samples = 25
np.random.seed(123)
random_samples = np.random.randint(0, 160, n_samples)
n_eig = 64
X_values = np.loadtxt('data/data/X_test_h1_100_01.csv', delimiter=',')
y_values = np.loadtxt('data/data/y_test_h1_100_01.csv', delimiter=',')

# Resolution Parameters for Different Solvers
resolutions = [(100, 100), (50, 50), (25, 25), (10, 10)]
field_mean, field_stdev, lamb_cov, mkl = 1, 1, 0.1, 64 

    ### Chapter one> how I overcame mz fear of codin

# Instantiate Models for Different Resolutions
solver_h1 = Model(resolutions[0], field_mean, field_stdev, mkl, lamb_cov)
solver_h2 = Model(resolutions[1], field_mean, field_stdev, mkl, lamb_cov)
solver_h3 = Model(resolutions[2], field_mean, field_stdev, mkl, lamb_cov)
solver_h4 = Model(resolutions[3], field_mean, field_stdev, mkl, lamb_cov)

def setup_random_process(solver_high, solver_low):
    """
    Synchronize the random processes between the higher and lower fidelity models
    by matching transmissivity fields across different resolutions.
    """
    coords_high = solver_high.solver.mesh.coordinates()
    coords_low = solver_low.solver.mesh.coordinates()
    
    structured_high = np.array(coords_high).view([('', coords_high.dtype)] * coords_high.shape[1])
    structured_low = np.array(coords_low).view([('', coords_low.dtype)] * coords_low.shape[1])
    
    bool_mask = np.in1d(structured_high, structured_low)
    solver_low.random_process.eigenvalues = solver_high.random_process.eigenvalues
    solver_low.random_process.eigenvectors = solver_high.random_process.eigenvectors[bool_mask]  



2025-05-26 17:39:22.537275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'model'

In [10]:
# Setup random processes between solvers
setup_random_process(solver_h1, solver_h2)
setup_random_process(solver_h1, solver_h3)
setup_random_process(solver_h1, solver_h4)

# Define Points for Data Extraction
x_data = y_data = np.array([0.1, 0.3, 0.5, 0.7, 0.9])
datapoints = np.array(list(product(x_data, y_data)))

# Solver Data Functions
def solver_h1_data(x):
    solver_h1.solve(x)
    return solver_h1.get_data(datapoints)
def solver_h2_data(x):
    solver_h2.solve(x) 
    return solver_h2.get_data(datapoints)
def solver_h3_data(x): 
    solver_h3.solve(x)
    return solver_h3.get_data(datapoints)
def solver_h4_data(x): 
    solver_h4.solve(x)
    return solver_h4.get_data(datapoints)

def model_HF(input): return solver_h1_data(input).flatten()
def model_LF1(input): return solver_h2_data(input).flatten()
def model_LF2(input): return solver_h3_data(input).flatten()
def model_LF3(input): return solver_h4_data(input).flatten()





In [13]:
case="1-step"
# Model Definitions for Different Cases
if case == "2-step":
    # Load Models for Low- and Multi-fidelity Predictions
        # Load Models for Low- and Multi-fidelity Predictions
    model_nn = [load_model(f'models/single_fidelity_100/resolution_10/samples_64000/model_fold_{i}.keras') for i in range(1,5)]
    models_1 = load_model(f'models/multi_fidelity_100_2step/input_10/samples_64000/model_fold_1.keras')
    models_2 = load_model(f'models/multi_fidelity_100_2step/input_10_25/samples_64000/model_fold_1.keras')
    models_3 = load_model(f'models/multi_fidelity_100_2step/input_10_25_50/samples_64000/model_fold_1.keras')
    
    @tf.function(jit_compile=True) 
    def model_mf1(input1, input2):
        input1    = tf.reshape(input1, (1, 64))
        input2    = tf.reshape(input2, (1, 25))
        return models_1([input1,input2], training=False)[0]
    
    @tf.function(jit_compile=True) 
    def model_mf2(input1, input2, input3):
        input1    = tf.reshape(input1, (1, 64))
        input2    = tf.reshape(input2, (1, 25))
        input3    = tf.reshape(input3, (1, 25))
        return models_2([input1,input2,input3], training=False)[0]
    
    @tf.function(jit_compile=True) 
    def model_mf3(input1, input2, input3, input4):
        input1    = tf.reshape(input1, (1, 64))
        input2    = tf.reshape(input2, (1, 25))
        input3    = tf.reshape(input3, (1, 25))
        input4    = tf.reshape(input4, (1, 25))
        return models_3([input1,input2,input3,input4], training=False)[0]
    
    # def model_1(input):
    #     coarse_data1 = tf.constant(solver_h4_data(input), dtype=tf.float32)
    #     return model_mf1(input, coarse_data1).numpy().flatten()
    @tf.function(jit_compile=True) 
    def model_cc(input):
        inputx = tf.reshape(input, (1, 64))
        return [Model(inputx, training=False)[0] for Model in model_nn]
    
    def model_1(input):
        res = model_cc(input)
        coarse_data1 = tf.constant(np.mean([tens.numpy() for tens in res],axis=0))
        return model_mf1(input, coarse_data1).numpy().flatten()
    
    def model_2(input):
        res = model_cc(input)
        coarse_data1 = tf.constant(np.mean([tens.numpy() for tens in res],axis=0))
        coarse_data2 = tf.constant(solver_h3_data(input), dtype=tf.float32)
        return model_mf2(input, coarse_data2, coarse_data1).numpy().flatten()

    def model_3(input):
        coarse_data2 = tf.constant(solver_h3_data(input), dtype=tf.float32)
        coarse_data3 = tf.constant(solver_h2_data(input), dtype=tf.float32)
        res = model_cc(input)
        coarse_data1 = tf.constant(np.mean([tens.numpy() for tens in res],axis=0))
        return model_mf3(input, coarse_data3, coarse_data2,coarse_data1).numpy().flatten()
    
    def model_HF(input): return solver_h1_data(input).flatten()
    
    NUM_DATAPOINTS = 64
    input_data = np.random.normal(size=(NUM_DATAPOINTS,1))
    input1 =  input_data 
    input2 = solver_h4_data(input1)
    input3 = solver_h3_data(input1)
    input4 = solver_h2_data(input1)
    execution_times = np.mean(repeat(lambda:  model_mf1(input1, input2,), number=1, repeat=500))
    print(execution_times)

elif case == "1-step":
    # Load Models for Low- and Multi-fidelity Predictions
    models_1 = load_model(f'models/multi_fidelity_100/input_10/samples_64000/model_fold_1.keras')
    models_2 = load_model(f'models/multi_fidelity_100/input_10_25/samples_64000/model_fold_1.keras')
    models_3 = load_model(f'models/multi_fidelity_100/input_10_25_50/samples_64000/model_fold_1.keras')

    # # Define TensorFlow Functions with JIT Compilation
    # @tf.function(jit_compile=True)
    # def model_lf(input):
    #     input_reshaped = tf.reshape(input, (1, 64))
    #     return tf.reduce_mean([mod(input_reshaped, training=False)[0] for mod in models_l], axis=0)

    @tf.function(jit_compile=True) 
    def model_mf1(input1, input2):
        input1    = tf.reshape(input1, (1, 64))
        input2    = tf.reshape(input2, (1, 25))
        return models_1([input1,input2], training=False)[0]
    
    @tf.function(jit_compile=True) 
    def model_mf2(input1, input2, input3):
        input1    = tf.reshape(input1, (1, 64))
        input2    = tf.reshape(input2, (1, 25))
        input3    = tf.reshape(input3, (1, 25))
        return models_2([input1,input2,input3], training=False)[0]
    
    @tf.function(jit_compile=True) 
    def model_mf3(input1, input2, input3, input4):
        input1    = tf.reshape(input1, (1, 64))
        input2    = tf.reshape(input2, (1, 25))
        input3    = tf.reshape(input3, (1, 25))
        input4    = tf.reshape(input4, (1, 25))
        return models_3([input1,input2,input3,input4], training=False)[0]
    
    # def model_1(input):
    #     coarse_data1 = tf.constant(solver_h4_data(input), dtype=tf.float32)
    #     return model_mf1(input, coarse_data1).numpy().flatten()
    
    def model_1(input):
        coarse_data1 = tf.constant(solver_h4_data(input), dtype=tf.float32)
        return model_mf1(input, coarse_data1).numpy().flatten()
    
    def model_2(input):
        coarse_data1 = tf.constant(solver_h4_data(input), dtype=tf.float32)
        coarse_data2 = tf.constant(solver_h3_data(input), dtype=tf.float32)
        return model_mf2(input, coarse_data2, coarse_data1).numpy().flatten()

    def model_3(input):
        coarse_data1 = tf.constant(solver_h4_data(input), dtype=tf.float32)
        coarse_data2 = tf.constant(solver_h3_data(input), dtype=tf.float32)
        coarse_data3 = tf.constant(solver_h2_data(input), dtype=tf.float32)
        return model_mf3(input, coarse_data3, coarse_data2,coarse_data1).numpy().flatten()
    
    def model_HF(input): return solver_h1_data(input).flatten()
    
    NUM_DATAPOINTS = 64
    input_data = np.random.normal(size=(NUM_DATAPOINTS,1))
    input1 =  input_data 
    input2 = solver_h4_data(input1)
    input3 = solver_h3_data(input1)
    input4 = solver_h2_data(input1)
    execution_times = np.mean(repeat(lambda:  model_mf3(input1, input2, input3, input4), number=1, repeat=500))
    print(execution_times)
    execution_times = np.mean(repeat(lambda:  model_mf2(input1, input2, input3), number=1, repeat=500))
    print(execution_times)
    execution_times = np.mean(repeat(lambda:  model_mf1(input1, input2,), number=1, repeat=500))
    print(execution_times)
    
elif case == "MLDA":
    
    model_1 = model_LF2
    model_2 = model_LF1
    model_3 = model_HF

# Prior and Proposal Distributions
x_distribution = stats.multivariate_normal(mean=np.zeros(64), cov=np.eye(64))
Times, Time_ESS, ESS, samples_tot, ERR = [], [], [], [], []

X_values = np.loadtxt('data/data/X_test_h1_100_01.csv', delimiter=',')
y_values = np.loadtxt('data/data/y_test_h1_100_01.csv', delimiter=',')

# Sampling for Each Random Sample
for i, sample in enumerate(random_samples, start=1):
    print(f'Sample = {sample}')
    x_true = X_values[sample]
    y_true = y_values[sample]
    y_observed = y_true + np.random.normal(scale=noise, size=y_true.shape[0])

    print(model_1(x_true))
    print(y_true)
    print(model_1(x_true).shape)
    print(y_true.shape)
    if case != "FOM":
        print(f"\nMSE coarse simulation 1 test:  {np.sqrt(np.mean((model_1(x_true) - y_true)**2)):.4e}")
        print(f"\nMSE coarse simulation 2 test:  {np.sqrt(np.mean((model_2(x_true) - y_true)**2)):.4e}")
        print(f"\nMSE coarse simulation 3 test:  {np.sqrt(np.mean((model_3(x_true) - y_true)**2)):.4e}")
        print(f"\nMSE coarse simulation HF test:  {np.sqrt(np.mean((model_HF(x_true) - y_values[sample])**2)):.4e}")
        
    
    def ls(x):
        return (y_true-model_HF(x))**2

    res = least_squares(ls,np.zeros_like((x_true)), jac='3-point')
    covariance = np.linalg.pinv(res.jac.T @ res.jac)
    covariance *= 1/np.max(np.abs(covariance))
    print(covariance[1:5,1:5])

    # Likelihood Distributions
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution_1 = tda.AdaptiveGaussianLogLike(y_observed, cov_likelihood*scaling1)
    y_distribution_2 = tda.AdaptiveGaussianLogLike(y_observed, cov_likelihood*scaling2)
    y_distribution_3 = tda.AdaptiveGaussianLogLike(y_observed, cov_likelihood*scaling3)
    y_distribution_fine = tda.GaussianLogLike(y_observed, cov_likelihood)
    my_proposal = tda.GaussianRandomWalk(C=covariance,scaling=1e-1, adaptive=True, gamma=1.1, period=10)
    # Initialize Posteriors
    my_posteriors = [
        tda.Posterior(x_distribution, y_distribution_1, model_1), 
        tda.Posterior(x_distribution, y_distribution_2, model_2),
        tda.Posterior(x_distribution, y_distribution_3, model_3)
    ] if case != "FOM" else tda.Posterior(x_distribution, y_distribution_fine, model_HF)

    # Run MCMC Sampling
    start_time = timeit.default_timer()
    samples = tda.sample(my_posteriors, my_proposal, iterations=n_iter, n_chains=1,
                            initial_parameters=res.x, subchain_length=sub_sampling,
                            adaptive_error_model='state-independent',store_coarse_chain=False)
    elapsed_time = timeit.default_timer() - start_time

    # Effective Sample Size (ESS)
    idata = tda.to_inference_data(samples, level=2).sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)
    mean_ess = np.mean([ess.data_vars[f'x{j}'].values for j in range(64)])

    # Store Results
    Times.append(elapsed_time)
    ESS.append(mean_ess)
    Time_ESS.append(elapsed_time / mean_ess)
    post = idata.posterior
    val=post.mean().to_array()
    err=(np.mean(np.sqrt((x_true-val)**2)))
    ERR.append(err)
    print(f'Time: {elapsed_time:.2f}, ESS: {mean_ess:.2f}, Time/ESS: {elapsed_time / mean_ess:.2f}, Err: {err:.3f} ({i}/{n_samples})')

# Save Results
output_folder = './data/recorded_values'
np.save(os.path.join(output_folder, f'MDA_MF_{case}_ratio_001.npy'), Time_ESS)
np.save(os.path.join(output_folder, f'MDA_MF_{case}_times_001.npy'), Times)
np.save(os.path.join(output_folder, f'MDA_MF_{case}_err_001.npy'), ERR)
np.save(os.path.join(output_folder, f'MDA_MF_{case}_ESS_001.npy'), ESS)

0.0009062300566583871
0.0008098156275227666
0.0007159502161666751
Sample = 109
[0.9564479  0.8841571  0.86788327 0.87858546 0.910012   0.8146133
 0.76014256 0.7290053  0.6728442  0.6965503  0.43945324 0.4523449
 0.4816191  0.56627077 0.5644229  0.22606157 0.23834382 0.28414515
 0.29701015 0.43436247 0.0821754  0.07835829 0.04931031 0.05931016
 0.07404209]
[0.96192542 0.88050663 0.86865022 0.88009208 0.91090491 0.81368249
 0.76461159 0.72644523 0.67516483 0.69961803 0.43187063 0.45375642
 0.48067429 0.56770859 0.56693449 0.21962048 0.23454837 0.28713743
 0.29760478 0.43583237 0.08227923 0.07619704 0.05019349 0.05727882
 0.0724779 ]
(25,)
(25,)

MSE coarse simulation 1 test:  3.0811e-03

MSE coarse simulation 2 test:  9.4146e-04

MSE coarse simulation 3 test:  1.3975e-04

MSE coarse simulation HF test:  0.0000e+00


KeyboardInterrupt: 